## 1. Image Data Handling

In [1]:
%pylab inline

import pandas as pd
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

import keras as K
from keras.models import Sequential
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, InputLayer, \
    Convolution2D, MaxPooling2D, Flatten,   \
    Dropout, BatchNormalization, GlobalAveragePooling2D

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
# To stop potential randomness
seed = 42
rng = np.random.RandomState(seed)

In [3]:
data = pd.read_csv('../datasets/male_vs_female_classification/male_vs_female_classification.csv')

In [4]:
data.head()

,image_name,class
0,1267.jpg,1
1,1268.jpg,1
2,1269.jpg,1
3,1270.jpg,1
4,1271.jpg,1


In [5]:
data['class'].value_counts()

1    8853
0    8570
Name: class, dtype: int64

## 2. Data Loading and Preprocessing

In [6]:
images = []
for img_loc in data.image_name:
    img = image.load_img('../datasets/male_vs_female_classification/images/' + img_loc)
    img = image.img_to_array(img)
    images.append(img)
    
images=np.array(images)

In [7]:
images.shape

(17423, 224, 224, 3)

In [8]:
train_x = images / images.max()
train_y = data['class'].values

In [9]:
X_train, X_valid, Y_train, Y_valid=train_test_split(train_x,train_y,test_size=0.3, random_state=42)

## 3. Model Building

In [10]:
model = Sequential([
 InputLayer(input_shape=(224, 224, 3)),

 Convolution2D(32, (5, 5), activation='relu', padding='same', name = "block1_conv1"),
 MaxPooling2D(pool_size=2, name = "block1_pool"),
 BatchNormalization(name = "block1_batchnorm"),

 Convolution2D(64, (5, 5), activation='relu', padding='same', name = "block2_conv1"),
 Convolution2D(64, (5, 5), activation='relu', padding='same', name = "block2_conv2"),
 MaxPooling2D(pool_size=2, name = "block2_pool"),
 BatchNormalization(name = "block2_batchnorm"),

 Convolution2D(128, (5, 5), activation='relu', padding='same', name = "block3_conv1"),
 Convolution2D(128, (5, 5), activation='relu', padding='same', name = "block3_conv2"),
 MaxPooling2D(pool_size=2, name = "block3_pool"),
 BatchNormalization(name = "block3_batchnorm"),
    
 Convolution2D(256, (5, 5), activation='relu', padding='same', name = "block4_conv1"),
 Convolution2D(256, (5, 5), activation='relu', padding='same', name = "block4_conv2"),
 MaxPooling2D(pool_size=2, name = "block4_pool"),
 BatchNormalization(name = "block4_batchnorm"),
    
 Convolution2D(512, (5, 5), activation='relu', padding='same', name = "block5_conv1"),
 Convolution2D(512, (5, 5), activation='relu', padding='same', name = "block5_conv2"),  
 MaxPooling2D(pool_size=2, name = "block5_pool"),


 GlobalAveragePooling2D(),
 BatchNormalization(),

 Dense(units=512, activation='sigmoid', name = 'fc1'),
 BatchNormalization(),
 Dropout(0.5),
    
 Dense(units=1, activation='sigmoid', name = 'predictions'),
])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 32)      2432      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 32)      0         
_________________________________________________________________
block1_batchnorm (BatchNorma (None, 112, 112, 32)      128       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 64)      51264     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 64)      102464    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_batchnorm (BatchNorma (None, 56, 56, 64)        256       
__________

In [12]:
datagen = image.ImageDataGenerator()
#     width_shift_range = 0.2,
#     horizontal_flip = True,
#     height_shift_range = 0.2,
#     rotation_range = 20
# )

In [13]:
final_weights_path = '../models/best_cnn_model.h5'

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=20, verbose=1)
]

In [14]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

In [15]:
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=256),epochs=100,validation_data=(X_valid,Y_valid), callbacks=callbacks_list, shuffle=False, workers=20, use_multiprocessing=True)

Epoch 1/100
48/48 [==============================] - 80s 2s/step - loss: 0.9975 - acc: 0.5470 - val_loss: 1.1092 - val_acc: 0.5091

Epoch 00001: val_loss improved from inf to 1.10923, saving model to ../models/best_cnn_model.h5
Epoch 2/100
48/48 [==============================] - 59s 1s/step - loss: 0.7202 - acc: 0.6384 - val_loss: 0.5810 - val_acc: 0.7084

Epoch 00002: val_loss improved from 1.10923 to 0.58098, saving model to ../models/best_cnn_model.h5
Epoch 3/100
48/48 [==============================] - 59s 1s/step - loss: 0.5743 - acc: 0.7295 - val_loss: 0.8810 - val_acc: 0.5799

Epoch 00003: val_loss did not improve from 0.58098
Epoch 4/100
48/48 [==============================] - 59s 1s/step - loss: 0.4965 - acc: 0.7697 - val_loss: 0.8868 - val_acc: 0.6964

Epoch 00004: val_loss did not improve from 0.58098
Epoch 5/100
48/48 [==============================] - 59s 1s/step - loss: 0.4064 - acc: 0.8124 - val_loss: 0.7282 - val_acc: 0.7207

Epoch 00005: val_loss did not improve from

Process ForkPoolWorker-311:
Process ForkPoolWorker-318:
Process ForkPoolWorker-314:
Process ForkPoolWorker-312:
Process ForkPoolWorker-308:
Process ForkPoolWorker-302:
Process ForkPoolWorker-310:
Process ForkPoolWorker-304:
Process ForkPoolWorker-301:
Process ForkPoolWorker-313:
Process ForkPoolWorker-307:
Process ForkPoolWorker-305:
Process ForkPoolWorker-316:
Process ForkPoolWorker-309:
Process ForkPoolWorker-320:
Process ForkPoolWorker-317:
Process ForkPoolWorker-306:
Process ForkPoolWorker-303:
Process ForkPoolWorker-319:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _boots

  File "/opt/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/opt/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
KeyboardInterrupt
  File "/opt/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", 

KeyboardInterrupt: 

In [16]:
model.load_weights(final_weights_path)

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 32)      2432      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 32)      0         
_________________________________________________________________
block1_batchnorm (BatchNorma (None, 112, 112, 32)      128       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 64)      51264     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 64)      102464    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_batchnorm (BatchNorma (None, 56, 56, 64)        256       
__________

In [18]:
for i in range(0,6):
    model.pop()

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 32)      2432      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 32)      0         
_________________________________________________________________
block1_batchnorm (BatchNorma (None, 112, 112, 32)      128       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 64)      51264     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 64)      102464    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_batchnorm (BatchNorma (None, 56, 56, 64)        256       
__________

/opt/anaconda/lib/python3.6/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [20]:
import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv2'

In [21]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [22]:
# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [23]:
kept_filters = []
for filter_index in range(128):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))



Processing filter 0
Current loss value: 10880.925
Current loss value: 11712.92
Current loss value: 12561.908
Current loss value: 13420.363
Current loss value: 14301.216
Current loss value: 15180.873
Current loss value: 16056.887
Current loss value: 16931.324
Current loss value: 17799.871
Current loss value: 18660.68
Current loss value: 19523.162
Current loss value: 20384.086
Current loss value: 21246.86
Current loss value: 22109.459
Current loss value: 22970.13
Current loss value: 23826.2
Current loss value: 24683.033
Current loss value: 25545.3
Current loss value: 26414.48
Current loss value: 27285.38
Filter 0 processed in 0s
Processing filter 1
Current loss value: 19148.1
Current loss value: 20184.178
Current loss value: 21263.336
Current loss value: 22365.0
Current loss value: 23510.027
Current loss value: 24665.502
Current loss value: 25820.307
Current loss value: 26960.574
Current loss value: 28087.309
Current loss value: 29208.0
Current loss value: 30309.457
Current loss value: 3

Current loss value: 16.130344
Current loss value: 41.294754
Current loss value: 75.226944
Current loss value: 118.236786
Current loss value: 171.7281
Current loss value: 238.88965
Current loss value: 316.2468
Current loss value: 397.75916
Current loss value: 480.3407
Current loss value: 565.2726
Current loss value: 655.11
Current loss value: 750.8903
Current loss value: 850.2317
Current loss value: 956.70154
Current loss value: 1068.4634
Current loss value: 1181.2532
Current loss value: 1297.0522
Current loss value: 1420.019
Current loss value: 1546.4175
Current loss value: 1675.4861
Filter 23 processed in 0s
Processing filter 24
Current loss value: 17331.92
Current loss value: 18476.965
Current loss value: 19654.44
Current loss value: 20879.385
Current loss value: 22149.342
Current loss value: 23425.63
Current loss value: 24711.229
Current loss value: 25995.102
Current loss value: 27276.445
Current loss value: 28556.715
Current loss value: 29830.379
Current loss value: 31103.934
Curre

Current loss value: 3790.5454
Current loss value: 3920.066
Current loss value: 4045.8403
Current loss value: 4163.8945
Current loss value: 4276.7534
Current loss value: 4387.6265
Current loss value: 4496.8667
Current loss value: 4606.977
Current loss value: 4709.9077
Current loss value: 4813.45
Current loss value: 4916.0293
Current loss value: 5014.9277
Current loss value: 5113.257
Current loss value: 5210.2686
Current loss value: 5308.8013
Current loss value: 5410.5723
Current loss value: 5510.469
Current loss value: 5608.709
Current loss value: 5713.4478
Current loss value: 5825.1035
Filter 38 processed in 1s
Processing filter 39
Current loss value: 7869.0127
Current loss value: 8710.133
Current loss value: 9563.011
Current loss value: 10443.816
Current loss value: 11335.158
Current loss value: 12221.709
Current loss value: 13110.996
Current loss value: 13999.45
Current loss value: 14885.011
Current loss value: 15778.559
Current loss value: 16668.93
Current loss value: 17562.762
Curr

Current loss value: 8970.265
Current loss value: 9507.836
Current loss value: 10079.196
Current loss value: 10698.381
Current loss value: 11348.449
Current loss value: 12007.003
Current loss value: 12672.375
Current loss value: 13348.104
Current loss value: 14025.854
Current loss value: 14701.122
Current loss value: 15379.218
Current loss value: 16064.463
Current loss value: 16753.5
Current loss value: 17443.754
Current loss value: 18139.791
Current loss value: 18841.877
Current loss value: 19543.451
Current loss value: 20242.129
Current loss value: 20944.266
Current loss value: 21652.098
Filter 54 processed in 1s
Processing filter 55
Current loss value: 8767.546
Current loss value: 9498.687
Current loss value: 10242.314
Current loss value: 11002.621
Current loss value: 11777.827
Current loss value: 12562.887
Current loss value: 13349.322
Current loss value: 14125.115
Current loss value: 14895.599
Current loss value: 15669.209
Current loss value: 16432.256
Current loss value: 17197.418

Current loss value: 0.0
Filter 73 processed in 1s
Processing filter 74
Current loss value: 5724.3877
Current loss value: 6188.425
Current loss value: 6658.864
Current loss value: 7142.9375
Current loss value: 7639.984
Current loss value: 8142.465
Current loss value: 8641.252
Current loss value: 9148.355
Current loss value: 9649.613
Current loss value: 10146.643
Current loss value: 10640.21
Current loss value: 11129.32
Current loss value: 11616.672
Current loss value: 12103.21
Current loss value: 12586.942
Current loss value: 13070.813
Current loss value: 13553.232
Current loss value: 14035.614
Current loss value: 14522.293
Current loss value: 15008.355
Filter 74 processed in 1s
Processing filter 75
Current loss value: 0.0
Filter 75 processed in 1s
Processing filter 76
Current loss value: 0.0
Filter 76 processed in 1s
Processing filter 77
Current loss value: 635.7956
Current loss value: 734.8
Current loss value: 826.04004
Current loss value: 902.6605
Current loss value: 988.75323
Curren

Current loss value: 0.0
Filter 99 processed in 1s
Processing filter 100
Current loss value: 4473.944
Current loss value: 4616.055
Current loss value: 4759.4824
Current loss value: 4899.1616
Current loss value: 5037.336
Current loss value: 5168.3887
Current loss value: 5299.248
Current loss value: 5422.0137
Current loss value: 5549.439
Current loss value: 5678.659
Current loss value: 5808.5156
Current loss value: 5936.3735
Current loss value: 6062.4077
Current loss value: 6191.9126
Current loss value: 6320.574
Current loss value: 6451.2725
Current loss value: 6582.7695
Current loss value: 6715.892
Current loss value: 6849.0
Current loss value: 6981.8154
Filter 100 processed in 1s
Processing filter 101
Current loss value: 6334.996
Current loss value: 6802.7
Current loss value: 7283.3774
Current loss value: 7768.905
Current loss value: 8261.383
Current loss value: 8752.631
Current loss value: 9237.059
Current loss value: 9720.511
Current loss value: 10201.743
Current loss value: 10680.934

Current loss value: 20091.79
Current loss value: 21466.684
Current loss value: 22878.428
Current loss value: 24339.277
Current loss value: 25855.523
Current loss value: 27378.43
Current loss value: 28897.46
Current loss value: 30419.219
Current loss value: 31941.246
Current loss value: 33459.496
Current loss value: 34973.43
Current loss value: 36479.91
Current loss value: 37993.875
Current loss value: 39498.184
Current loss value: 40992.625
Current loss value: 42487.938
Current loss value: 43985.867
Current loss value: 45480.184
Current loss value: 46978.383
Current loss value: 48474.74
Filter 120 processed in 1s
Processing filter 121
Current loss value: 6882.837
Current loss value: 7472.1953
Current loss value: 8057.625
Current loss value: 8639.739
Current loss value: 9233.189
Current loss value: 9833.072
Current loss value: 10426.822
Current loss value: 11007.599
Current loss value: 11576.15
Current loss value: 12135.51
Current loss value: 12693.096
Current loss value: 13247.582
Curr

In [24]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img



In [25]:
# save the result to disk
save_img('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

In [ ]:
predictions = model.predict_classes(X_valid).ravel()
prediction_probabilities = model.predict(X_valid).ravel()
all_vehicles = np.array(emergency_vehicles + nonemergency_vehicles)

_, valid_vehicles, _, valid_y = train_test_split(all_vehicles,train_y,test_size=0.3, random_state=42)

In [ ]:
correct_predictions = valid_vehicles[predictions == Y_valid]

index = rng.choice(range(len(correct_predictions)))
img_name = correct_predictions[index]


prob = (prediction_probabilities[predictions == Y_valid] * 100).astype(int)[index]

img = imread('../datasets/emergency_classification/images/' + img_name)

print(prob , '% sure that it is emergency')
pylab.imshow(img)
pylab.axis('off')
pylab.show()

In [ ]:
incorrect_predictions = valid_vehicles[predictions != Y_valid]
index = rng.choice(range(len(incorrect_predictions)))
img_name = incorrect_predictions[index]


prob = (prediction_probabilities[predictions != Y_valid] * 100).astype(int)[index]

img = imread('../datasets/emergency_classification/images/' + img_name)

print(prob , '% sure that it is emergency')
pylab.imshow(img)
pylab.axis('off')
pylab.show()

In [ ]:
print(classification_report(predictions, Y_valid))

In [ ]:
tn, fp, fn, tp = confusion_matrix(predictions, Y_valid).ravel()

In [ ]:
tn, tp, fn, fp

In [ ]:
roc_auc_score(predictions, Y_valid)

In [ ]:
top_layer = model.layers[0]
plt.imshow(top_layer.get_weights()[0][:, :, :, 0].squeeze())
plt.show()

In [ ]:
min_val = np.abs(top_layer.get_weights()[0][:, :, :, 0].squeeze().min())

In [ ]:
min_val

In [ ]:
plt.imshow(top_layer.get_weights()[0][:, :, :, 0].squeeze() + min_val)

In [ ]:
from vis.visualization import visualize_activation
from vis.utils import utils
from keras import activations

plt.rcParams['figure.figsize'] = (18, 6)

layer_idx = -1

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear

# This is the output node we want to maximize.
filter_idx = 0
img = visualize_activation(model, layer_idx, filter_indices=filter_idx)
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
from vis.visualization import get_num_filters

# The name of the layer we want to visualize
# You can see this in the model definition.
layer_idx = 4

# Visualize all filters in this layer.
filters = np.arange(get_num_filters(model.layers[layer_idx]))

# Generate input image for each filter.
vis_images = []
for idx in filters[:4]:
    img = visualize_activation(model, layer_idx, filter_indices=idx)
    
    # Utility to overlay text on image.
    img = utils.draw_text(img, 'Filter {}'.format(idx))    
    vis_images.append(img)

# Generate stitched image palette with 8 cols.
stitched = utils.stitch_images(vis_images, cols=8)    
plt.axis('off')
plt.imshow(stitched)
plt.title(layer_idx)
plt.show()

In [ ]:
img_name = rng.choice(emergency_vehicles)

img = imread('../datasets/emergency_classification/images/' +img_name)

In [ ]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam
from vis.visualization import visualize_saliency, overlay


plt.figure()
f, ax = plt.subplots()
plt.suptitle("vanilla")

grads = visualize_cam(model, -1, filter_indices=0, 
                      seed_input=img, backprop_modifier=None)        
# Lets overlay the heatmap onto original image.    
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
ax.imshow(overlay(jet_heatmap, img))

In [ ]:
for img_name in emergency_vehicles:
    img = imread('../datasets/emergency_classification/images/' + img_name)
    
    plt.figure()
    f, ax = plt.subplots()
    plt.suptitle("vanilla")

    grads = visualize_cam(model, -1, filter_indices=0, 
                          seed_input=img, backprop_modifier=None)        
    # Lets overlay the heatmap onto original image.    
    jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
    ax.imshow(overlay(jet_heatmap, img))
    plt.show()

In [ ]:
vehicles = glob('../datasets/emergency_classification/images/*.jpg')

In [ ]:
img_name = vehicles[285]
img = imread(img_name)

plt.figure()
f, ax = plt.subplots()
plt.suptitle("vanilla")

grads = visualize_cam(model, -1, filter_indices=0, 
                      seed_input=img, backprop_modifier=None)        
# Lets overlay the heatmap onto original image.    
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
ax.imshow(overlay(jet_heatmap, img))
plt.show()